# Deploy a spark model from an environment

## First get the Workspace and register the model

In [0]:
import azureml
from azureml.core import Workspace
import mlflow.azureml
from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.environment import Environment

workspace_name = '<YOUR-WORKSPACE-NAME>'
resource_group = '<YOUR-RESOURCE-GROUP>'
subscription_id = '<YOUR-SUBSCRIPTION-ID>'

def get_workspace(workspace_name, resource_group, subscription_id):
  svc_pr = ServicePrincipalAuthentication(
      tenant_id = dbutils.secrets.get(scope = "azure-key-vault", key = "tenant-id"),
      service_principal_id = dbutils.secrets.get(scope = "azure-key-vault", key = "cliente-id-custom-role"),
      service_principal_password = dbutils.secrets.get(scope = "azure-key-vault", key = "cliente-secret-custom-role"))

  workspace = Workspace.get(name = workspace_name,
                            resource_group = resource_group,
                            subscription_id = subscription_id,
                            auth=svc_pr)
  
  return workspace

workspace = get_workspace(workspace_name, resource_group, subscription_id)

### Entry script

In [0]:
%%writefile /tmp/score.py

import os
import pickle
import json
import time
from mlflow.pyfunc import load_model
import pandas as pd
import traceback

# Called when the deployed service starts
def init():
    global model
    # Get the path where the deployed model can be found.
    model_path = os.getenv('AZUREML_MODEL_DIR') + '/your-model'
    # load models
    model = load_model(model_path)

def run(data):
    try:
        data = pd.DataFrame(json.loads(data))
        prediction = model.predict(data)
        return {"prediction": str(prediction)}
    except Exception as e:
        traceback.print_exc()
        error = str(e)
        return error

### Register a new model version

In [0]:
from azureml.core.model import Model

model_name = 'model-spark'
model_path = '/dbfs/mnt/models/your-model'
model_description = 'My spark model'

model_azure = Model.register(model_path=model_path,
                             model_name=model_name,
                             description=model_description,
                             workspace=workspace,
                             tags={'Framework': "Spark"})

### Define an inference config

In [0]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

my_spark_env = Environment.get(name='spark-env-custom', workspace=workspace)

inference_config = InferenceConfig(entry_script="/tmp/score.py", environment=my_spark_env)

## Deploy the model

### ACI

In [0]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

service_name = 'api-model-dev'

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description="This is a spark serving example.")
 
service = Model.deploy(name=service_name, deployment_config=aci_config, models=[model_azure], inference_config=inference_config, workspace=workspace, overwrite=True)
service.wait_for_deployment(show_output=True)